<a href="https://colab.research.google.com/github/IsaacFigNewton/Analyzing-Hate-Crime-Data/blob/main/Hate_Crime_Data_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TODO:
####Replace Anti-Church of Jesus Christ bias with Anti-Mormon
####Add county demographic data from other years
####Get p-values for in merged_df?

#Import Stuff

##Import all libraries

In [1]:
import pandas as pd
import sklearn as sk
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import re
import warnings

##Config

In [2]:
warnings.filterwarnings(action='ignore')

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
minimum_categorical_positive_count = 5
default_z_threshold_for_outliers = 2.5
# years to draw crime data from
years = [i for i in range(2010, 2020)] + [2021, 2022] #[2022]  #

##Import datasets

In [5]:
# use the entries of the first row as the column headers for easier management
def fixHeaders(df):
  new_headers = df.iloc[0]
  new_df = df[1:]
  new_df.columns = new_headers
  return new_df

In [6]:
crime_df = pd.read_csv("https://raw.githubusercontent.com/IsaacFigNewton/Analyzing-Hate-Crime-Data/main/datasets/hate_crime/hate_crime.csv", on_bad_lines='skip')

In [7]:
# create the county_demo_df
county_demo_df = pd.DataFrame()

# combine all datasets of the specified years
for year in years:
  temp_df = pd.read_csv("https://raw.githubusercontent.com/IsaacFigNewton/Analyzing-Hate-Crime-Data/main/datasets/demographics/county/data/ACSDP1Y" + str(year) + ".DP05-Data.csv", on_bad_lines='skip')

  #fix the headers
  temp_df = fixHeaders(temp_df)
  temp_df["data_year"] = year

  print(year)

  if county_demo_df.empty:
    county_demo_df = temp_df
  else:
    # drop all column duplicates
    county_demo_df = county_demo_df.loc[:,~county_demo_df.columns.duplicated()].copy()
    temp_df = temp_df.loc[:,~temp_df.columns.duplicated()].copy()

    cols_to_merge_on = list(set(temp_df.columns).intersection(set(county_demo_df.columns)))
    county_demo_df = pd.merge(county_demo_df, temp_df, how="outer")

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2021
2022


In [8]:
ethnicity_race_cols = list(set(pd.read_csv("https://raw.githubusercontent.com/IsaacFigNewton/Analyzing-Hate-Crime-Data/main/datasets/demographics/county/ethnicity_race_col_names", sep=";")["Percent!!RACE!!One race"]))
# ethnicity_race_cols

In [9]:
county_demo_df.head()

,Geography,Geographic Area Name,Estimate!!SEX AND AGE!!Total population,Estimate Margin of Error!!SEX AND AGE!!Total population,Percent!!SEX AND AGE!!Total population,Percent Margin of Error!!SEX AND AGE!!Total population,Estimate!!SEX AND AGE!!Male,Estimate Margin of Error!!SEX AND AGE!!Male,Percent!!SEX AND AGE!!Male,Percent Margin of Error!!SEX AND AGE!!Male,Estimate!!SEX AND AGE!!Female,Estimate Margin of Error!!SEX AND AGE!!Female,Percent!!SEX AND AGE!!Female,Percent Margin of Error!!SEX AND AGE!!Female,Estimate!!SEX AND AGE!!Under 5 years,Estimate Margin of Error!!SEX AND AGE!!Under 5 years,Percent!!SEX AND AGE!!Under 5 years,Percent Margin of Error!!SEX AND AGE!!Under 5 years,Estimate!!SEX AND AGE!!5 to 9 years,Estimate Margin of Error!!SEX AND AGE!!5 to 9 years,Percent!!SEX AND AGE!!5 to 9 years,Percent Margin of Error!!SEX AND AGE!!5 to 9 years,Estimate!!SEX AND AGE!!10 to 14 years,Estimate Margin of Error!!SEX AND AGE!!10 to 14 years,Percent!!SEX AND AGE!!10 to 14 years,Percent Margin of Error!!SEX AND AGE!!10 to 14 years,Estimate!!SEX AND AGE!!15 to 19 years,Estimate Margin of Error!!SEX AND AGE!!15 to 19 years,Percent!!SEX AND AGE!!15 to 19 years,Percent Margin of Error!!SEX AND AGE!!15 to 19 years,Estimate!!SEX AND AGE!!20 to 24 years,Estimate Margin of Error!!SEX AND AGE!!20 to 24 years,Percent!!SEX AND AGE!!20 to 24 years,Percent Margin of Error!!SEX AND AGE!!20 to 24 years,Estimate!!SEX AND AGE!!25 to 34 years,Estimate Margin of Error!!SEX AND AGE!!25 to 34 years,Percent!!SEX AND AGE!!25 to 34 years,Percent Margin of Error!!SEX AND AGE!!25 to 34 years,Estimate!!SEX AND AGE!!35 to 44 years,Estimate Margin of Error!!SEX AND AGE!!35 to 44 years,Percent!!SEX AND AGE!!35 to 44 years,Percent Margin of Error!!SEX AND AGE!!35 to 44 years,Estimate!!SEX AND AGE!!45 to 54 years,Estimate Margin of Error!!SEX AND AGE!!45 to 54 years,Percent!!SEX AND AGE!!45 to 54 years,Percent Margin of Error!!SEX AND AGE!!45 to 54 years,Estimate!!SEX AND AGE!!55 to 59 years,Estimate Margin of Error!!SEX AND AGE!!55 to 59 years,Percent!!SEX AND AGE!!55 to 59 years,Percent Margin of Error!!SEX AND AGE!!55 to 59 years,Estimate!!SEX AND AGE!!60 to 64 years,Estimate Margin of Error!!SEX AND AGE!!60 to 64 years,Percent!!SEX AND AGE!!60 to 64 years,Percent Margin of Error!!SEX AND AGE!!60 to 64 years,Estimate!!SEX AND AGE!!65 to 74 years,Estimate Margin of Error!!SEX AND AGE!!65 to 74 years,Percent!!SEX AND AGE!!65 to 74 years,Percent Margin of Error!!SEX AND AGE!!65 to 74 years,Estimate!!SEX AND AGE!!75 to 84 years,Estimate Margin of Error!!SEX AND AGE!!75 to 84 years,Percent!!SEX AND AGE!!75 to 84 years,Percent Margin of Error!!SEX AND AGE!!75 to 84 years,Estimate!!SEX AND AGE!!85 years and over,Estimate Margin of Error!!SEX AND AGE!!85 years and over,Percent!!SEX AND AGE!!85 years and over,Percent Margin of Error!!SEX AND AGE!!85 years and over,Estimate!!SEX AND AGE!!Median age (years),Estimate Margin of Error!!SEX AND AGE!!Median age (years),Percent!!SEX AND AGE!!Median age (years),Percent Margin of Error!!SEX AND AGE!!Median age (years),Estimate!!SEX AND AGE!!18 years and over,Estimate Margin of Error!!SEX AND AGE!!18 years and over,Percent!!SEX AND AGE!!18 years and over,Percent Margin of Error!!SEX AND AGE!!18 years and over,Estimate!!SEX AND AGE!!21 years and over,Estimate Margin of Error!!SEX AND AGE!!21 years and over,Percent!!SEX AND AGE!!21 years and over,Percent Margin of Error!!SEX AND AGE!!21 years and over,Estimate!!SEX AND AGE!!62 years and over,Estimate Margin of Error!!SEX AND AGE!!62 years and over,Percent!!SEX AND AGE!!62 years and over,Percent Margin of Error!!SEX AND AGE!!62 years and over,Estimate!!SEX AND AGE!!65 years and over,Estimate Margin of Error!!SEX AND AGE!!65 years and over,Percent!!SEX AND AGE!!65 years and over,Percent Margin of Error!!SEX AND AGE!!65 years and over,Estimate!!RACE!!Total population,Estimate Margin of Error!!RACE!!Total population,Percent!!RACE!!Total population,Percent Margin of Error!!RACE!!T

#Data Cleaning

In [10]:
#only consider county data from 2002-2022; had to expand scope since sample sizes were unreliably small using only 2022
crime_df = crime_df[(crime_df['data_year'].isin(years)) & ((crime_df['agency_type_name'] == "County"))]

In [11]:
crime_df.reset_index(inplace=True)

##Important Functions

In [12]:
def get_col_counts(df, unique_cols):
    # get the frequencies of categorical data based
    unique_col_counts = {}
    for value in unique_cols:
        if value in set(df.columns):
          unique_col_counts[value] = df[value].sum()
          # print(value)

    return unique_col_counts

In [13]:
def drop_outliers(df, numerical_cols, z_threshold=default_z_threshold_for_outliers):
    df_out = df.copy()  # Create a copy to avoid modifying the original DataFrame

    for column in numerical_cols:
        # Calculate Z-score for each value in the column
        z_scores = np.abs((df_out[column] - df_out[column].mean()) / df_out[column].std())

        print(z_scores.head())

        # Identify outliers based on Z-score
        outliers = df_out[z_scores > z_threshold]

        # Drop outliers from the DataFrame
        df_out = df_out.drop(outliers.index)

    return df_out

##Clean county demographic dataset

###Create area columns

In [14]:
def split_area_name(area):
    result = [np.nan, np.nan, np.nan]
    if ", " in area:
        result = area.split(", ") + [np.nan]
    if " city" in result[0].lower():
        result[0] = result[0][0:-5]
        result[2] = result[1]
        result[1] = "City"
    elif " county" in result[0].lower():
        result[0] = result[0][0:-7]
        result[2] = result[1]
        result[1] = "County"

    return result + [np.nan] * (3 - len(result))

def splitArea(df):
    df[["pug_agency_name", "agency_type_name", "state_name"]] = df["Geographic Area Name"].map(split_area_name).apply(pd.Series)

In [15]:
splitArea(county_demo_df)

###Fix column names



In [16]:
def fix_column_names(df):
  removalList = {"SEX AND AGE!!", "SEX AND ", "Estimate!!", "Total!!", "Total population!!", "CITIZEN, VOTING AGE POPULATION!!", "AGE!!"}

  new_cols = []
  for col in df.columns:
    col = str(col)
    for term in removalList:
      if term in col:
        col = col.replace(term, "")
    new_cols.append(col)

  return new_cols

county_demo_df.columns = fix_column_names(county_demo_df)

###Remove irrelevant/bad columns and rows to line the dataframes up

In [17]:
# fix any missing values
county_demo_df = county_demo_df.replace("(X)", np.nan)

In [18]:
#drop all margin of error columns from the county dataframe
def drop_cols_containing(df, pattern):
    df.drop(columns=list(df.filter(regex = pattern)), inplace = True)

drop_cols_containing(county_demo_df, "Margin of Error|SUMMARY INDICATORS|PERCENT ALLOCATED")

In [19]:
# Drop columns that only contain NaN values
new_county_df = county_demo_df.dropna(how='all', axis=1)

# get a list of the cols in the original df that aren't in the new one
dropped_cols = (set(county_demo_df.columns)).difference(set(new_county_df.columns))

# remove the deleted columns from ethnicity_race_cols
ethnicity_race_cols = list((set(ethnicity_race_cols)).difference(set(dropped_cols)))

county_demo_df = new_county_df

print(ethnicity_race_cols)

# Drop rows with NaN values in any of the ethnicity_race_cols
county_demo_df.dropna(subset=ethnicity_race_cols, how='any', inplace=True)

['Percent!!RACE!!One race!!Asian!!Asian Indian', 'Percent!!RACE!!One race!!Asian', 'Percent!!HISPANIC OR LATINO AND RACE!!Not Hispanic or Latino!!Two or More Races', 'Percent!!Citizen, 18 and over population!!Female', 'Percent!!RACE!!One race!!Some Other Race', 'Percent!!RACE!!One race!!Asian!!Korean', 'Percent!!RACE!!Two or More Races!!White and Asian', 'Percent!!RACE!!Two or More Races!!Black or African American and American Indian and Alaska Native', 'Percent!!HISPANIC OR LATINO AND RACE!!Total population', 'Percent!!RACE!!Two or More Races!!White and Some Other Race', 'Percent!!Race alone or in combination with one or more other races!!Native Hawaiian and Other Pacific Islander', 'Percent!!RACE!!One race!!Native Hawaiian and Other Pacific Islander!!Chamorro', 'Percent!!RACE!!One race!!White', 'Percent!!Race alone or in combination with one or more other races!!Some Other Race', 'Percent!!RACE!!Two or More Races!!White and American Indian and Alaska Native', 'Percent!!HISPANIC OR LA

###Fix column data types

In [20]:
counts = county_demo_df.columns.value_counts()
mask = counts > 1
duplicates = list(counts[mask].index)
county_demo_df[duplicates].dtypes

65 years and over!!Female                                                                                                                        object
65 years and over!!Female                                                                                                                        object
RACE!!Two or more races!!Black or African American and American Indian and Alaska Native                                                         object
RACE!!Two or more races!!Black or African American and American Indian and Alaska Native                                                         object
RACE!!Two or more races!!White and Black or African American                                                                                     object
RACE!!Two or more races!!White and Black or African American                                                                                     object
Percent!!RACE!!Two or more races!!White and Black or African American                   

In [21]:
county_demo_df = county_demo_df.loc[:, ~county_demo_df.columns.duplicated(keep='last')]

In [22]:
county_percent_age_cols = list(set(list(county_demo_df.filter(regex = "Percent"))\
                                  + list(county_demo_df.filter(regex = "ratio"))\
                                  + list(county_demo_df.filter(regex = "years"))))

for column in county_percent_age_cols:
    contaminated_indices_mask = county_demo_df[column]\
                                  .astype(str)\
                                  .str\
                                  .contains('N')
    county_demo_df.loc[contaminated_indices_mask, column] = 0

In [23]:
county_demo_df[county_percent_age_cols] = county_demo_df[county_percent_age_cols].astype(float)

In [24]:
county_demo_df[ethnicity_race_cols] = county_demo_df[ethnicity_race_cols].astype(float)

In [25]:
# Convert the quantitative columns to int types
# continuous quantitative variables
county_continuous_int_columns = ["Total population", "Under 5 years", "5 to 9 years", "45 to 54 years", "10 to 14 years", "75 to 84 years", "60 to 64 years",\
                      "25 to 34 years", "15 to 19 years", "20 to 24 years", "35 to 44 years", "55 to 59 years", "65 to 74 years",\
                      "85 years and over"] + ethnicity_race_cols

county_demo_df[county_continuous_int_columns].replace("N", 0, inplace=True)


county_demo_df[county_continuous_int_columns] = county_demo_df[county_continuous_int_columns].astype(int)

In [26]:
age_groups = ['Under 5 years', '5 to 9 years', '10 to 14 years', '15 to 19 years', '20 to 24 years', '25 to 34 years', '35 to 44 years', '45 to 54 years', '55 to 59 years', '60 to 64 years', '65 to 74 years', '75 to 84 years', '85 years and over']

###Drop outliers

In [27]:
county_demo_df = drop_outliers(county_demo_df, county_continuous_int_columns)

Series([], Name: Total population, dtype: float64)
Series([], Name: Under 5 years, dtype: float64)
Series([], Name: 5 to 9 years, dtype: float64)
Series([], Name: 45 to 54 years, dtype: float64)
Series([], Name: 10 to 14 years, dtype: float64)
Series([], Name: 75 to 84 years, dtype: float64)
Series([], Name: 60 to 64 years, dtype: float64)
Series([], Name: 25 to 34 years, dtype: float64)
Series([], Name: 15 to 19 years, dtype: float64)
Series([], Name: 20 to 24 years, dtype: float64)
Series([], Name: 35 to 44 years, dtype: float64)
Series([], Name: 55 to 59 years, dtype: float64)
Series([], Name: 65 to 74 years, dtype: float64)
Series([], Name: 85 years and over, dtype: float64)
Series([], Name: Percent!!RACE!!One race!!Asian!!Asian Indian, dtype: float64)
Series([], Name: Percent!!RACE!!One race!!Asian, dtype: float64)
Series([], Name: Percent!!HISPANIC OR LATINO AND RACE!!Not Hispanic or Latino!!Two or More Races, dtype: float64)
Series([], Name: Percent!!Citizen, 18 and over populat

In [28]:
county_demo_df["Geography"].count()

0

##Clean crime dataset

In [29]:
crime_continuous_int_columns = ["index", "incident_id", "data_year"] + list(crime_df.select_dtypes("float64").columns)

In [30]:
crime_df[crime_continuous_int_columns].head()

,index,incident_id,data_year,adult_victim_count,juvenile_victim_count,adult_offender_count,juvenile_offender_count,total_individual_victims
0,143544,143579,2010,NaN,NaN,NaN,NaN,1.0
1,143547,143582,2010,NaN,NaN,NaN,NaN,2.0
2,143555,143590,2010,NaN,NaN,NaN,NaN,1.0
3,143556,143591,2010,NaN,NaN,NaN,NaN,1.0
4,143557,143592,2010,NaN,NaN,NaN,NaN,1.0


In [31]:
crime_continuous_int_columns

['index',
 'incident_id',
 'data_year',
 'adult_victim_count',
 'juvenile_victim_count',
 'adult_offender_count',
 'juvenile_offender_count',
 'total_individual_victims']

###Drop outliers and bad rows

In [32]:
crime_df.head()

,index,incident_id,data_year,ori,pug_agency_name,pub_agency_unit,agency_type_name,state_abbr,state_name,division_name,region_name,population_group_code,population_group_description,incident_date,adult_victim_count,juvenile_victim_count,total_offender_count,adult_offender_count,juvenile_offender_count,offender_race,offender_ethnicity,victim_count,offense_name,total_individual_victims,location_name,bias_desc,victim_types,multiple_offense,multiple_bias
0,143544,143579,2010,AL0080000,Blount,NaN,County,AL,Alabama,East South Central,South,9B,"MSA counties from 25,000 thru 99,999",2010-06-03,NaN,NaN,1,NaN,NaN,White,Not Specified,1,Intimidation,1.0,Residence/Home,Anti-Hispanic or Latino,Individual,S,S
1,143547,143582,2010,AL0160000,Clarke,NaN,County,AL,Alabama,East South Central,South,8C,"Non-MSA counties from 10,000 thru 24,999",2010-02-18,NaN,NaN,4,NaN,NaN,Black or African American,Not Specified,2,Murder and Nonnegligent Manslaughter,2.0,Residence/Home,Anti-Other Religion,Individual,S,S
2,143555,143590,2010,AL0630000,Tuscaloosa,NaN,County,AL,Alabama,East South Central,South,9B,"MSA counties from 25,000 thru 99,999",2010-09-01,NaN,NaN,1,NaN,NaN,White,Not Specified,1,Intimidation,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-Black or African American,Individual,S,S
3,143556,143591,2010,AL0630000,Tuscaloosa,NaN,County,AL,Alabama,East South Central,South,9B,"MSA counties from 25,000 thru 99,999",2010-06-16,NaN,NaN,1,NaN,NaN,Black or African American,Not Specified,1,Simple Assault,1.0,Residence/Home,Anti-White,Individual,S,S
4,143557,143592,2010,AL0630000,Tuscaloosa,NaN,County,AL,Alabama,East South Central,South,9B,"MSA counties from 25,000 thru 99,999",2010-06-16,NaN,NaN,1,NaN,NaN,White,Not Specified,1,Simple Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S


In [33]:
# crime_df.reset_index(inplace=True)
crime_df = drop_outliers(crime_df, crime_continuous_int_columns)

0    1.579044
1    1.578942
2    1.578669
3    1.578635
4    1.578601
Name: index, dtype: float64
0    0.922841
1    0.922836
2    0.922823
3    0.922822
4    0.922820
Name: incident_id, dtype: float64
0    1.576171
1    1.576171
2    1.576171
3    1.576171
4    1.576171
Name: data_year, dtype: float64
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: adult_victim_count, dtype: float64
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: juvenile_victim_count, dtype: float64
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: adult_offender_count, dtype: float64
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: juvenile_offender_count, dtype: float64
0    0.035438
1    0.894430
2    0.035438
3    0.035438
4    0.035438
Name: total_individual_victims, dtype: float64


In [34]:
crime_df.head()

,index,incident_id,data_year,ori,pug_agency_name,pub_agency_unit,agency_type_name,state_abbr,state_name,division_name,region_name,population_group_code,population_group_description,incident_date,adult_victim_count,juvenile_victim_count,total_offender_count,adult_offender_count,juvenile_offender_count,offender_race,offender_ethnicity,victim_count,offense_name,total_individual_victims,location_name,bias_desc,victim_types,multiple_offense,multiple_bias
0,143544,143579,2010,AL0080000,Blount,NaN,County,AL,Alabama,East South Central,South,9B,"MSA counties from 25,000 thru 99,999",2010-06-03,NaN,NaN,1,NaN,NaN,White,Not Specified,1,Intimidation,1.0,Residence/Home,Anti-Hispanic or Latino,Individual,S,S
1,143547,143582,2010,AL0160000,Clarke,NaN,County,AL,Alabama,East South Central,South,8C,"Non-MSA counties from 10,000 thru 24,999",2010-02-18,NaN,NaN,4,NaN,NaN,Black or African American,Not Specified,2,Murder and Nonnegligent Manslaughter,2.0,Residence/Home,Anti-Other Religion,Individual,S,S
2,143555,143590,2010,AL0630000,Tuscaloosa,NaN,County,AL,Alabama,East South Central,South,9B,"MSA counties from 25,000 thru 99,999",2010-09-01,NaN,NaN,1,NaN,NaN,White,Not Specified,1,Intimidation,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-Black or African American,Individual,S,S
3,143556,143591,2010,AL0630000,Tuscaloosa,NaN,County,AL,Alabama,East South Central,South,9B,"MSA counties from 25,000 thru 99,999",2010-06-16,NaN,NaN,1,NaN,NaN,Black or African American,Not Specified,1,Simple Assault,1.0,Residence/Home,Anti-White,Individual,S,S
4,143557,143592,2010,AL0630000,Tuscaloosa,NaN,County,AL,Alabama,East South Central,South,9B,"MSA counties from 25,000 thru 99,999",2010-06-16,NaN,NaN,1,NaN,NaN,White,Not Specified,1,Simple Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S


###Break up incident date information

In [35]:
def split_incident_date(date):
    result = date.split("-")

    return result + [np.nan] * (3 - len(result))

crime_df[["data_year", "incident_month", "incident_day"]] = crime_df["incident_date"]\
                                                              .map(split_incident_date)\
                                                              .apply(pd.Series)\
                                                              .astype(int)

In [36]:
crime_df[["data_year", "incident_month", "incident_day"]].dtypes

data_year         int64
incident_month    int64
incident_day      int64
dtype: object

###Break up crime_df categorical columns containing multiple categories into dummies

In [37]:
def multicol_parse(df, column, delimiter):
  prefix = column + "_"

  dummies = df[column]\
              .str\
              .get_dummies(sep=delimiter)\
              .add_prefix(prefix)
  dummies = dummies.astype(int)
  dummies.head()

  dummy_cols = list(dummies.columns)
  unique_values = [val.replace(prefix, "") for val in dummy_cols]

  # combine the dataframes
  new_df = pd.concat([df, dummies], axis=1)

  return unique_values, dummy_cols, new_df

In [38]:
crime_df.reset_index(inplace=True)

In [39]:
#Break up bias_desc column into dummy columns
unique_biases, bias_cols, crime_df = multicol_parse(crime_df, "bias_desc", ";")

In [40]:
#Break up victim_types column into dummy columns
unique_victim_types, victim_cols, crime_df = multicol_parse(crime_df, "victim_types", ";")

In [41]:
#Break up offense_names column into dummy columns
unique_offense_names, offense_cols, crime_df = multicol_parse(crime_df, "offense_name", ";")

In [42]:
#Break up location_names column into dummy columns
unique_location_names, location_cols, crime_df = multicol_parse(crime_df, "location_name", ";")

###Create dummy columns for other categorical variables

In [43]:
other_categorical_cols = ["pug_agency_name", "agency_type_name", "division_name", "offender_race",
                          "offender_ethnicity", "state_name", "multiple_offense", "multiple_bias"]

In [44]:
other_categorical_cols

['pug_agency_name',
 'agency_type_name',
 'division_name',
 'offender_race',
 'offender_ethnicity',
 'state_name',
 'multiple_offense',
 'multiple_bias']

In [45]:
crime_df.head()

,level_0,index,incident_id,data_year,ori,pug_agency_name,pub_agency_unit,agency_type_name,state_abbr,state_name,division_name,region_name,population_group_code,population_group_description,incident_date,adult_victim_count,juvenile_victim_count,total_offender_count,adult_offender_count,juvenile_offender_count,offender_race,offender_ethnicity,victim_count,offense_name,total_individual_victims,location_name,bias_desc,victim_types,multiple_offense,multiple_bias,incident_month,incident_day,bias_desc_Anti-American Indian or Alaska Native,bias_desc_Anti-Arab,bias_desc_Anti-Asian,bias_desc_Anti-Atheism/Agnosticism,bias_desc_Anti-Bisexual,bias_desc_Anti-Black or African American,bias_desc_Anti-Buddhist,bias_desc_Anti-Catholic,bias_desc_Anti-Church of Jesus Christ,"bias_desc_Anti-Eastern Orthodox (Russian, Greek, Other)",bias_desc_Anti-Female,bias_desc_Anti-Gay (Male),bias_desc_Anti-Gender Non-Conforming,bias_desc_Anti-Heterosexual,bias_desc_Anti-Hindu,bias_desc_Anti-Hispanic or Latino,bias_desc_Anti-Islamic (Muslim),bias_desc_Anti-Jehovah's Witness,bias_desc_Anti-Jewish,bias_desc_Anti-Lesbian (Female),"bias_desc_Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)",bias_desc_Anti-Male,bias_desc_Anti-Mental Disability,"bias_desc_Anti-Multiple Races, Group","bias_desc_Anti-Multiple Religions, Group",bias_desc_Anti-Native Hawaiian or Other Pacific Islander,bias_desc_Anti-Other Christian,bias_desc_Anti-Other Race/Ethnicity/Ancestry,bias_desc_Anti-Other Religion,bias_desc_Anti-Physical Disability,bias_desc_Anti-Protestant,bias_desc_Anti-Sikh,bias_desc_Anti-Transgender,bias_desc_Anti-White,victim_types_Business,victim_types_Financial Institution,victim_types_Government,victim_types_Individual,victim_types_Law Enforcement Officer,victim_types_Other,victim_types_Religious Organization,victim_types_Society/Public,victim_types_Unknown,offense_name_Aggravated Assault,offense_name_All Other Larceny,offense_name_Animal Cruelty,offense_name_Arson,offense_name_Bribery,offense_name_Burglary/Breaking & Entering,offense_name_Counterfeiting/Forgery,offense_name_Credit Card/Automated Teller Machine Fraud,offense_name_Destruction/Damage/Vandalism of Property,offense_name_Drug Equipment Violations,offense_name_Drug/Narcotic Violations,offense_name_Embezzlement,offense_name_Extortion/Blackmail,offense_name_False Pretenses/Swindle/Confidence Game,offense_name_Fondling,offense_name_Hacking/Computer Invasion,"offense_name_Human Trafficking, Involuntary Servitude",offense_name_Identity Theft,offense_name_Impersonation,offense_name_Incest,offense_name_Intimidation,offense_name_Kidnapping/Abduction,offense_name_Motor Vehicle Theft,offense_name_Murder and Nonnegligent Manslaughter,offense_name_Not Specified,offense_name_Pocket-picking,offense_name_Pornography/Obscene Material,offense_name_Prostitution,offense_name_Purchasing Prostitution,offense_name_Purse-snatching,offense_name_Rape,offense_name_Robbery,offense_name_Sexual Assault With An Object,offense_name_Shoplifting,offense_name_Simple Assault,offense_name_Sodomy,offense_name_Statutory Rape,offense_name_Stolen Property Offenses,offense_name_Theft From Building,offense_name_Theft From Motor Vehicle,offense_name_Theft of Motor Vehicle Parts or Accessories,offense_name_Weapon Law Violations,offense_name_Welfare Fraud,offense_name_Wire Fraud,location_name_ATM Separate from Bank,location_name_Abandoned/Condemned Structure,location_name_Air/Bus/Train Terminal,location_name_Amusement Park,location_name_Arena/Stadium/Fairgrounds/Coliseum,location_name_Auto Dealership New/Used,location_name_Bank/Savings and Loan,location_name_Bar/Nightclub,location_name_Camp/Campground,location_name_Church/Synagogue/Temple/Mosque,location_name_Commercial/Office Building,location_name_Community Center,location_name_Construction Site,location_name_Convenience Store,location_name_Cyberspace,location_name_Daycare Facility,location_name_Department/Discount Store,location_name_Dock/Wharf/Freight/Modal Terminal,location_name_Drug Store/D

In [46]:
other_unique_categorical_cols = {}
for col in other_categorical_cols:
    unique_values = crime_df[col].unique()
    # print(unique_values)
    other_unique_categorical_cols[col] = [col + "_" + str(val) for val in unique_values]
print(other_unique_categorical_cols["pug_agency_name"])

['pug_agency_name_Blount', 'pug_agency_name_Clarke', 'pug_agency_name_Tuscaloosa', 'pug_agency_name_Benton', 'pug_agency_name_Garland', 'pug_agency_name_Independence', 'pug_agency_name_Lonoke', 'pug_agency_name_Pulaski', 'pug_agency_name_Perry', 'pug_agency_name_White', 'pug_agency_name_Maricopa', 'pug_agency_name_Pima', 'pug_agency_name_Pinal', 'pug_agency_name_Butte', 'pug_agency_name_Contra Costa', 'pug_agency_name_Kern', 'pug_agency_name_Lake', 'pug_agency_name_Los Angeles', 'pug_agency_name_Madera', 'pug_agency_name_Mendocino', 'pug_agency_name_Orange', 'pug_agency_name_Riverside', 'pug_agency_name_Sacramento', 'pug_agency_name_San Bernardino', 'pug_agency_name_San Diego', 'pug_agency_name_Santa Barbara', 'pug_agency_name_Santa Clara', 'pug_agency_name_Santa Cruz', 'pug_agency_name_Shasta', 'pug_agency_name_Stanislaus', 'pug_agency_name_Sonoma', 'pug_agency_name_Colusa', 'pug_agency_name_El Dorado', 'pug_agency_name_San Joaquin', 'pug_agency_name_Mariposa', 'pug_agency_name_Monter

In [47]:
dummies = pd.get_dummies(crime_df[other_categorical_cols]).astype(int)
dummies.drop(columns=['multiple_offense_S', 'multiple_bias_S'], inplace=True)
other_unique_categorical_cols["multiple_offense"] = ["multiple_offense_M"]
other_unique_categorical_cols["multiple_bias"] = ["multiple_bias_M"]

# combine the dummy columns with crime_df
crime_df = pd.concat([crime_df, dummies], axis=1)

###List the categorical columns

In [48]:
# break up other_unique_categorical_cols because Python is being stupid
crime_categorical_cols = bias_cols\
                          + victim_cols\
                          + offense_cols\
                          + location_cols
for key, value in other_unique_categorical_cols.items():
  crime_categorical_cols += value

#Data Engineering

##Perform power transformations and normalize data

###Transform data to Gaussian distributions

In [49]:
def transform_standardize_data(df, numerical_cols, categorical_cols):
  # increase subsampling size if df is bigger than the default size
  n_quantiles = len(df)
  subsample = 10000
  if n_quantiles > subsample:
    subsample = n_quantiles * 10

  print("Performing quantile transformation process...")
  # use a quantile transformer to normalize the data and since the PowerTransformer keeps shitting itself
  scaler = sk.preprocessing.QuantileTransformer(output_distribution="normal", n_quantiles=n_quantiles, subsample=subsample)
  transformed_data = scaler.fit_transform(df[numerical_cols])

  print("Renaming numerical columns...")
  for i, col in enumerate(numerical_cols):
    new_column_header = "transformed_" + col

    # update the list of numerical column labels
    numerical_cols[i] = new_column_header
    # add the transformed, standardized data back to the original dataframe
    df[new_column_header] = transformed_data[:, i]

  print("Dropping categorical columns with insufficient sample size...")
  # identify columns with insufficient sample sizes
  columns_to_drop = []
  for col in categorical_cols:
      if df[col].sum() < minimum_categorical_positive_count:
          columns_to_drop.append(col)

  # drop identified columns
  df.drop(columns_to_drop, axis=1, inplace=True)

  # update categorical_cols after dropping
  categorical_cols = [col for col in categorical_cols if col not in columns_to_drop]

In [50]:
crime_numerical_cols = list(crime_df.select_dtypes(include=['int64', 'float64']).columns)
county_numerical_cols = list(county_demo_df.select_dtypes(include=["int64", "float64"]).columns)

In [51]:
# yeo-johnson transformation is super brittle and I'm not sure why
#transform the county data
transform_standardize_data(county_demo_df, county_numerical_cols, [])

Performing quantile transformation process...


InvalidParameterError: The 'n_quantiles' parameter of QuantileTransformer must be an int in the range [1, inf). Got 0 instead.

In [ ]:
#transform the non-categorical data of crime_df
transform_standardize_data(crime_df,\
                            [col for col in crime_numerical_cols\
                                      if not col in crime_categorical_cols],\
                            crime_categorical_cols)

In [ ]:
# remove columns that were removed from crime_df due to insufficient sample size from consideration
county_numerical_cols = [col for col in county_numerical_cols if col in set(county_demo_df.columns)]

# remove columns that were removed from crime_df due to insufficient sample size from consideration
crime_numerical_cols = [col for col in crime_numerical_cols if col in set(crime_df.columns)]

In [ ]:
print(crime_numerical_cols)

In [ ]:
# [col for col in crime_numerical_cols if not col in crime_categorical_cols]

##Extract additional features from the data

####Get hate crime rate per county

In [ ]:
crime_df.head()

In [ ]:
# print(set([col for col in crime_df.columns if "pug_agency_name" in col]))

# Remove agencies that were dropped from the list of unique categorical columns
other_unique_categorical_cols["pug_agency_name"] = list((set(other_unique_categorical_cols["pug_agency_name"])).intersection(set(crime_df.columns)))
# print(other_unique_categorical_cols["pug_agency_name"])

In [ ]:
print(other_unique_categorical_cols)

In [ ]:
county_hate_counts = get_col_counts(crime_df, other_unique_categorical_cols["pug_agency_name"])
county_hate_rates_df = pd.DataFrame.from_dict(data=county_hate_counts, orient='index', columns=['hate_crime_count'])

In [ ]:
county_hate_rates_df["pug_agency_name"] = county_hate_rates_df\
                                            .index\
                                            .str\
                                            .replace("pug_agency_name_", "")

In [ ]:
county_hate_rates_df["pug_agency_name"].head()

In [ ]:
# only include demographic info related to each of the counties in the crime dataset
filtered_county_demo_df = county_demo_df[(county_demo_df["pug_agency_name"]).isin(county_hate_rates_df["pug_agency_name"])]
filtered_county_demo_df["Geography"].count()

In [ ]:
# merge the 2 datasets
county_hate_rates_df = county_hate_rates_df.merge(filtered_county_demo_df, on="pug_agency_name", how="outer")

In [ ]:
# filtered_county_demo_df.head()

In [ ]:
county_hate_rates_df = county_hate_rates_df.dropna(subset=['Geography'])

In [ ]:
county_hate_rates_df["hate_crime_rate"] = county_hate_rates_df["hate_crime_count"]\
                                            .divide(county_hate_rates_df["Total population"])\
                                            .astype("float64")

In [ ]:
county_hate_rates_df.head()

In [ ]:
county_hate_rates_df["hate_crime_rate"].plot.hist(bins=20)

####Transform hate_rate data to a Gaussian distribution

In [ ]:
hate_features = ["hate_crime_count", "hate_crime_rate"]
# save the hate feature data for adding back later
hate_crime_rates_df = county_hate_rates_df[hate_features]

# transform the hate_crime_rate data
transform_standardize_data(county_hate_rates_df, hate_features, [])
county_hate_rates_df = pd.concat([county_hate_rates_df, hate_crime_rates_df], axis=1, join="outer")

hate_rate_numerical_cols = [col for col in county_hate_rates_df.select_dtypes(include=["int64", "float64"]).columns if (("transformed" in col)\
                                                                                                                        or (col in hate_features))]

county_hate_rates_df = county_hate_rates_df[hate_rate_numerical_cols]

In [ ]:
county_hate_rates_df['transformed_hate_crime_rate'].plot.hist(bins=100)

##Merge the datasets

In [ ]:
# combine the crime and population datasets
merged_df = pd.merge(crime_df, county_demo_df, on=["pug_agency_name", "agency_type_name", "state_name", "data_year"])

In [ ]:
merged_df.drop(other_unique_categorical_cols["pug_agency_name"] + ["ori", "state_abbr", "population_group_code", "pub_agency_unit",
                "Geography", "Percent!!Total population", "population_group_description",
                "Geographic Area Name", "incident_date", "pub_agency_unit", "pug_agency_name",
                "agency_type_name", "state_name", "division_name", "offender_race", "offender_ethnicity",
                "location_name", "bias_desc", "victim_types", "multiple_offense",
                  "multiple_bias"], axis=1, inplace=True)

#Data Exploration

##Important Functions

In [ ]:
def get_sorted_corrs(corr):
  # Unstack and rename corr
  corr_sorted = pd.DataFrame(corr.unstack()).reset_index()
  corr_sorted.rename(columns={"level_0":"feature1", "level_1":"feature2", 0:"corr"}, inplace=True)

  corr_sorted["abs_corr"] = 0

  # Get highest correlation coefficient pairs
  corr_sorted["abs_corr"] = corr_sorted["corr"].abs()
  corr_sorted.sort_values(by="abs_corr", ascending=False, inplace=True)

  #reset the indices
  corr_sorted.reset_index(inplace=True)

  #filter out self-correlations
  corr_sorted = corr_sorted[~(corr_sorted["feature1"] == corr_sorted["feature2"])]

  # remove duplicates
  # only keep even-numbered rows, since there will be 2 of every correlation
  corr_sorted = corr_sorted.iloc[::2]
  # corr_sorted = corr_sorted.dropna().groupby("feature1")

  return corr_sorted

In [ ]:
def do_corr_analysis(corr, minThreshold, maxThreshold, blacklistPhraseRegex, corr_sorted=None):
    # get the sorted correlation coefficients
    if corr_sorted is None:
      sorted_corrs = get_sorted_corrs(crime_corr)
    else:
      sorted_corrs = corr_sorted

    # create subplot display
    fig, ax = plt.subplots(3, figsize=(10, 20))

    # plot stuff
    sorted_corrs["corr"].plot.hist(bins=40, logy=True, ax=ax[0], title="Correlation Coefficients")
    sorted_corrs["abs_corr"].plot.hist(bins=40, logy=True, ax=ax[1], title="Correlation Coefficients' Absolute Values")

    # filter down to correlations of interest
    corrs_of_interest = sorted_corrs[(sorted_corrs["abs_corr"] > minThreshold) & (sorted_corrs["abs_corr"] < maxThreshold)
                                      & (~sorted_corrs["feature1"].str.contains(blacklistPhraseRegex))
                                      & (~sorted_corrs["feature2"].str.contains(blacklistPhraseRegex))]

    # plot the remaining correlations
    corrs_of_interest["corr"].plot.hist(bins=50, logy=True, ax=ax[2], title="Correlation Coefficients of Interest")

    return sorted_corrs, corrs_of_interest

##Individual Datasets

###Hate Crime Dataset

In [ ]:
crime_df.head()

####Correlation matrix

In [ ]:
for item in ['level_0', 'index', 'incident_id', 'data_year']:
  if item in crime_numerical_cols:
    crime_numerical_cols.remove(item)

In [ ]:
# Create a correlation matrix
crime_corr = crime_df[crime_numerical_cols].dropna().corr()

In [ ]:
# # Set the figure size
# plt.figure(figsize=(200, 100))

# # Create a heatmap of the correlation matrix
# sns.heatmap(crime_corr, annot=True)

# # Show the plot
# plt.show()

In [ ]:
sorted_crime_corrs, crime_corrs_of_interest = do_corr_analysis(crime_corr, 0.1, 1, "pug_agency_name|state_name")

In [ ]:
crime_corrs_of_interest

####Quantitative variable figures

In [ ]:
crime_df['total_offender_count'].plot.hist(bins=14, title='total_offender_count', logy=True)

In [ ]:
crime_df['adult_victim_count'].plot.hist(bins=15, title='adult_victim_count', logy=True)

In [ ]:
crime_df['juvenile_victim_count'].plot.hist(bins=10, title='juvenile_victim_count', logy=True)

####Categorical variable figures


In [ ]:
def plot_column_counts(df, column_name, unique_values, title_phrase, figsize=(20, 5), rotation=90):
    # Create a dictionary of unique values and their counts
    unique_value_counts = get_col_counts(df, unique_values)

    # Sort the dictionary by values in descending order
    sorted_value_counts = dict(sorted(unique_value_counts.items(), key=lambda item: item[1], reverse=True))

    # Create a bar chart of the sorted valuees and their counts
    plt.figure(figsize=figsize)
    sns.barplot(x=list(sorted_value_counts.keys()), y=list(sorted_value_counts.values()), log=True)

    # Set the title and labels
    plt.title("Frequency of " + title_phrase + " in Hate Crimes", fontsize=16)
    plt.xlabel(title_phrase, fontsize=14)
    plt.ylabel("Frequency (Log Scale)", fontsize=14)

    # Rotate the x-axis labels for readability
    plt.xticks(rotation=rotation)

    # Show the plot
    plt.show()


In [ ]:
crime_df.groupby('region_name').size().sort_values(ascending=False).plot.bar(color=sns.palettes.mpl_palette('Dark2'))
plt.xticks(rotation=0)

In [ ]:
crime_df.groupby('offender_race').size().sort_values(ascending=True).plot.barh(color=sns.palettes.mpl_palette('Dark2'), figsize=(10,10), logx=True)

In [ ]:
plot_column_counts(crime_df, "bias_desc", bias_cols, "Biases")

In [ ]:
figsize=(20, 5)
plot_column_counts(crime_df, "victim_types", victim_cols, "Victims", figsize, 0)

In [ ]:
plot_column_counts(crime_df, "offense_name", offense_cols, "Offenses")

In [ ]:
plot_column_counts(crime_df, "location_names", location_cols, "Locations")

####Geographic heat maps

In [ ]:
ignore_list = ["Not Specified", "Unknown", "Multiple"]

In [ ]:
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['offender_race'].value_counts()
    for x_label, grp in crime_df.groupby('region_name')
})

# Drop less relevant columns for easier comparison with victim race heatmap
for item in ignore_list:
  df_2dhist.drop(item, inplace=True)

# Apply logarithmic transformation to the counts
df_2dhist_log = df_2dhist.applymap(lambda x: 0 if x == 0 else np.log10(x))

sns.heatmap(df_2dhist_log, cmap='viridis')
plt.title("Logarithmic Frequency of Offender Races by Region")
plt.xlabel("Region Name")
plt.ylabel("Offender Race")

In [ ]:
# Get some victim races to start with
unique_offender_races = crime_df['offender_race'].unique().tolist()

# Drop categories absent from victim data
for item in ignore_list:
  unique_offender_races.remove(item)

unique_victim_races = ["bias_desc_Anti-" + str(race) for race in unique_offender_races]

print(unique_victim_races)

In [ ]:
plt.subplots(figsize=(8, 8))

# Initialize an empty DataFrame with the correct index and columns
df_2dhist = pd.DataFrame(index=unique_victim_races, columns=crime_df['region_name'].unique())

# Iterate over each region and calculate the value counts for each victim race
for region in df_2dhist.columns:
    region_data = crime_df[crime_df['region_name'] == region]

    for victim_race in unique_victim_races:
        count = 0
        # Sum of occurrences of victim_race in the region
        if victim_race in region_data.columns:
          count = region_data[victim_race].sum()
        # Update the DataFrame cell with the count
        df_2dhist.loc[victim_race, region] = count

# Convert DataFrame entries to numeric type and drop nan values
df_2dhist = df_2dhist.apply(pd.to_numeric)

# Apply logarithmic transformation to the counts
df_2dhist = df_2dhist.applymap(lambda x: 0 if x == 0 else np.log10(x))

# Create the heatmap with logarithmic scale
sns.heatmap(df_2dhist, cmap='viridis')

# Give the plot a title and axis labels
plt.title("Logarithmic Frequency of Victim Races by Region")
plt.xlabel("Region Name")
plt.ylabel("Victim Races")
plt.yticks(rotation=0)

plt.show()

###County Demographics Dataset

In [ ]:
county_demo_df.head()

####Correlation matrix

In [ ]:
county_numerical_cols = list(county_demo_df.select_dtypes(include=["int64", "float64"]).columns)

In [ ]:
# Create a correlation matrix
county_corr = county_demo_df[county_numerical_cols].corr()

In [ ]:
# # Set the figure size
# plt.figure(figsize=(100, 50))

# # Create a heatmap of the correlation matrix
# sns.heatmap(county_corr, annot=True)

# # Show the plot
# plt.show()

In [ ]:
sorted_county_corrs, county_corrs_of_interest = do_corr_analysis(county_corr, 0.2, 0.9, "pug_agency_name|state_name")

In [ ]:
county_corrs_of_interest

####Other figures

In [ ]:
county_demo_df.plot.scatter(x='Under 5 years', y='85 years and over', logx=True, logy=True, s=40, alpha=.8, figsize=(16,8))

In [ ]:
county_demo_df[age_groups].mean().plot.bar(figsize=(20,5))
plt.xlabel('Age Group')
plt.ylabel('Mean Population')
plt.title('Mean Population by Age Group in City Demographics')
plt.xticks(rotation=0)
plt.show()

##Merged Datasets

###Primary Merged Dataset

In [ ]:
merged_df.head()

####Correlation Matrix Analysis

In [ ]:
#convert the categorical columns to numerical ones and store the modified df as a new df for correlation analysis
categorical_to_int_df = merged_df\
                          .select_dtypes(include=['int64', 'float64', np.number])\
                          .drop(labels=["index", "incident_id"], axis=1)
# Select only the numerical columns
numerical_cols = [col for col in categorical_to_int_df.columns if "transformed_" in col]
# numerical_cols

In [ ]:
# Create a correlation matrix
merged_corr = merged_df[numerical_cols].corr()

In [ ]:
merged_corr_sorted = get_sorted_corrs(merged_corr)
merged_corr_sorted.head()

In [ ]:
# remove coefficients that represent variables' correlations with themselves
#   and remove feature correlations with other features from the same set (which were reviewed in the individual dataset explorations above)
county_cols = list(county_numerical_cols)
crime_cols = list(crime_numerical_cols)

merged_corr_sorted = merged_corr_sorted[~(
    ((merged_corr_sorted["feature1"].isin(county_cols))
      & (merged_corr_sorted["feature2"].isin(county_cols)))
    | ((merged_corr_sorted["feature1"].isin(crime_cols))
      & (merged_corr_sorted["feature2"].isin(crime_cols)))
)]

In [ ]:
merged_corr_sorted.head()

In [ ]:
# drop half the entries, which are essentially duplicates,
#   as well as any np.nan entries and entries with low correlation coefficients or exceedingly high correlation coefficients,
#   which have already been considered previously or present trivial correlation information
#   (ie the number of individuals under 5 years of age is directly correlated with the total population)
merged_corr_sorted, merged_corrs_of_interest = do_corr_analysis(merged_corr, 0.1, 1, "state_name|division_name|pug_agency_name|index|incident_id", merged_corr_sorted)

In [ ]:
merged_corrs_of_interest

In [ ]:
merged_corrs_of_interest.iloc[0]["feature1"]

####Other Figures

In [ ]:
merged_df.plot.scatter(x="transformed_Percent!!RACE!!One race!!White", y="transformed_Percent!!45 to 54 years")

###County Feature Prediction Dataset

In [ ]:
county_hate_rates_df.head()

####Correlation Analysis

In [ ]:
hate_rate_corr = county_hate_rates_df[hate_rate_numerical_cols].corr()

In [ ]:
hate_rate_corr.head()

In [ ]:
# # Set the figure size
# plt.figure(figsize=(100, 50))

# # Create a heatmap of the correlation matrix
# sns.heatmap(hate_rate_corr, annot=True)

# # Show the plot
# plt.show()

In [ ]:
extracted_sorted_corrs, extracted_corrs_of_interest = do_corr_analysis(hate_rate_corr["transformed_hate_crime_rate"], 0.1, 1.01, "state_name|division_name|pug_agency_name")

In [ ]:
# look at just the correlations between the hate crime rate and other features
hate_rate_corr.loc["transformed_hate_crime_rate"].sort_values(ascending=False)

####Scatter Plots

In [ ]:
county_hate_rates_df.plot.scatter(x="transformed_Percent!!Race alone or in combination with one or more other races!!White", y="transformed_hate_crime_rate")

In [ ]:
county_hate_rates_df.plot.scatter(x="transformed_Percent!!RACE!!Two or More Races", y="transformed_hate_crime_rate")

In [ ]:
county_hate_rates_df.plot.scatter(x="transformed_Percent!!Race alone or in combination with one or more other races!!American Indian and Alaska Native", y="transformed_hate_crime_rate")

In [ ]:
county_hate_rates_df.plot.scatter(x="transformed_Under 5 years", y="transformed_hate_crime_rate")

In [ ]:
county_hate_rates_df.plot.scatter(x="transformed_Percent!!RACE!!One race!!White", y="transformed_hate_crime_rate")

In [ ]:
county_hate_rates_df.plot.scatter(x="transformed_Percent!!RACE!!One race", y="transformed_hate_crime_rate")

#Data Prediction

In [ ]:
# designate the training data
training_cols = [col for col in hate_rate_numerical_cols if (not "hate_crime_rate" in col) and ("transformed_" in col)]
x_train_df = county_hate_rates_df[training_cols]
y_train_df = county_hate_rates_df["transformed_hate_crime_rate"]

# designate a single feature for viewing the model's predictions relative to its transformed value
review_feature = "transformed_Percent!!Race alone or in combination with one or more other races!!White"

In [ ]:
x_train_df.head()

##Linear Regression on All Transformed Data

In [ ]:
hate_regression_model = LinearRegression()
hate_regression_model.fit(
    X=x_train_df.dropna(),
    y=y_train_df.dropna()
)

####Observe regression line for individual feature

In [ ]:
coefficients = hate_regression_model.coef_
intercept = hate_regression_model.intercept_

In [ ]:
# create a scatter plot of the data
county_hate_rates_df.plot.scatter(x=review_feature, y="transformed_hate_crime_rate")

x = x_train_df[review_feature]
plt.plot(x, coefficients[x_train_df.columns.get_loc(review_feature)] * x)

##K-Nearest-Neighbors on Review Feature

In [ ]:
# reformat the training data
x_train_df = pd.DataFrame(x_train_df[review_feature]).dropna()
y_train_df.dropna(inplace=True)

# create the test data
x_test = pd.DataFrame()
x_test[review_feature] = np.linspace(-2, 5, 1000)

# try various different values of k
for k in range(50, len(x_train_df), 50):
    # create and fit the model
    hate_knn_model = KNeighborsRegressor(n_neighbors=k)
    hate_knn_model.fit(X=x_train_df, y=y_train_df)

    # get the model's predictions
    y_pred = pd.Series(
        hate_knn_model.predict(x_test),
        index=x_test[review_feature]
    )

    # plot knn model predictions
    y_pred.plot.line(label=("k = " + str(k))).legend()

# create a scatter plot of the data
county_hate_rates_df.plot.scatter(x=review_feature, y="transformed_hate_crime_rate")